# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 15 2022 3:26PM,241998,19,DEX0007194,Dexcel Pharma Technologies Ltd. DBA WDSrx,Executing
1,Jul 15 2022 3:26PM,241998,19,DEX0007195,Dexcel Pharma Technologies Ltd. DBA WDSrx,Executing
2,Jul 15 2022 3:26PM,241998,19,DEX0007196,Dexcel Pharma Technologies Ltd. DBA WDSrx,Executing
3,Jul 15 2022 3:26PM,241998,19,DEX0007197,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
4,Jul 15 2022 3:26PM,241998,19,DEX0007198,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
5,Jul 15 2022 2:58PM,241993,10,0085877555,ISDIN Corporation,Released
6,Jul 15 2022 2:58PM,241993,10,0085877562,ISDIN Corporation,Released
7,Jul 15 2022 2:58PM,241993,10,0085877566,ISDIN Corporation,Released
8,Jul 15 2022 2:58PM,241993,10,0085877565,ISDIN Corporation,Released
9,Jul 15 2022 2:58PM,241993,10,0085877570,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,241989,Released,1
15,241991,Released,1
16,241993,Released,10
17,241998,Executing,3
18,241998,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
241988,NaN,5.0
241989,NaN,1.0
241991,NaN,1.0
241993,NaN,10.0
241998,3.0,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
241918,38.0,36.0
241924,1.0,0.0
241925,0.0,1.0
241926,0.0,1.0
241976,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
241918,38,36
241924,1,0
241925,0,1
241926,0,1
241976,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,241918,38,36
1,241924,1,0
2,241925,0,1
3,241926,0,1
4,241976,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,241918,38,36
1,241924,1,
2,241925,,1
3,241926,,1
4,241976,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 15 2022 3:26PM,241998,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
5,Jul 15 2022 2:58PM,241993,10,ISDIN Corporation
15,Jul 15 2022 2:28PM,241991,10,"ClearSpec, LLC"
16,Jul 15 2022 2:26PM,241989,10,Eye Pharma Inc
17,Jul 15 2022 2:25PM,241988,10,"Methapharm, Inc."
22,Jul 15 2022 2:20PM,241987,10,"Methapharm, Inc."
65,Jul 15 2022 2:15PM,241986,10,Yusen – 3D Matrix
66,Jul 15 2022 2:05PM,241985,10,"ClearSpec, LLC"
67,Jul 15 2022 1:53PM,241984,20,Alumier Labs Inc.
89,Jul 15 2022 1:44PM,241983,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 15 2022 3:26PM,241998,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,3,2
1,Jul 15 2022 2:58PM,241993,10,ISDIN Corporation,,10
2,Jul 15 2022 2:28PM,241991,10,"ClearSpec, LLC",,1
3,Jul 15 2022 2:26PM,241989,10,Eye Pharma Inc,,1
4,Jul 15 2022 2:25PM,241988,10,"Methapharm, Inc.",,5
5,Jul 15 2022 2:20PM,241987,10,"Methapharm, Inc.",,43
6,Jul 15 2022 2:15PM,241986,10,Yusen – 3D Matrix,,1
7,Jul 15 2022 2:05PM,241985,10,"ClearSpec, LLC",,1
8,Jul 15 2022 1:53PM,241984,20,Alumier Labs Inc.,,22
9,Jul 15 2022 1:44PM,241983,10,"Citieffe, Inc.",2,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 15 2022 3:26PM,241998,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2,3
1,Jul 15 2022 2:58PM,241993,10,ISDIN Corporation,10,
2,Jul 15 2022 2:28PM,241991,10,"ClearSpec, LLC",1,
3,Jul 15 2022 2:26PM,241989,10,Eye Pharma Inc,1,
4,Jul 15 2022 2:25PM,241988,10,"Methapharm, Inc.",5,
5,Jul 15 2022 2:20PM,241987,10,"Methapharm, Inc.",43,
6,Jul 15 2022 2:15PM,241986,10,Yusen – 3D Matrix,1,
7,Jul 15 2022 2:05PM,241985,10,"ClearSpec, LLC",1,
8,Jul 15 2022 1:53PM,241984,20,Alumier Labs Inc.,22,
9,Jul 15 2022 1:44PM,241983,10,"Citieffe, Inc.",,2


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 15 2022 3:26PM,241998,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2,3
1,Jul 15 2022 2:58PM,241993,10,ISDIN Corporation,10,
2,Jul 15 2022 2:28PM,241991,10,"ClearSpec, LLC",1,
3,Jul 15 2022 2:26PM,241989,10,Eye Pharma Inc,1,
4,Jul 15 2022 2:25PM,241988,10,"Methapharm, Inc.",5,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 15 2022 3:26PM,241998,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2.0,3.0
1,Jul 15 2022 2:58PM,241993,10,ISDIN Corporation,10.0,NaN
2,Jul 15 2022 2:28PM,241991,10,"ClearSpec, LLC",1.0,NaN
3,Jul 15 2022 2:26PM,241989,10,Eye Pharma Inc,1.0,NaN
4,Jul 15 2022 2:25PM,241988,10,"Methapharm, Inc.",5.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 15 2022 3:26PM,241998,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2.0,3.0
1,Jul 15 2022 2:58PM,241993,10,ISDIN Corporation,10.0,0.0
2,Jul 15 2022 2:28PM,241991,10,"ClearSpec, LLC",1.0,0.0
3,Jul 15 2022 2:26PM,241989,10,Eye Pharma Inc,1.0,0.0
4,Jul 15 2022 2:25PM,241988,10,"Methapharm, Inc.",5.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1935902,62.0,2.0
16,725775,2.0,1.0
19,483974,4.0,3.0
20,725883,59.0,38.0
21,241979,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1935902,62.0,2.0
1,16,725775,2.0,1.0
2,19,483974,4.0,3.0
3,20,725883,59.0,38.0
4,21,241979,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,62.0,2.0
1,16,2.0,1.0
2,19,4.0,3.0
3,20,59.0,38.0
4,21,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,62.0
1,16,Released,2.0
2,19,Released,4.0
3,20,Released,59.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,21
Status,,,,,
Executing,2.0,1.0,3.0,38.0,0.0
Released,62.0,2.0,4.0,59.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,21
0,Executing,2.0,1.0,3.0,38.0,0.0
1,Released,62.0,2.0,4.0,59.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,21
0,Executing,2.0,1.0,3.0,38.0,0.0
1,Released,62.0,2.0,4.0,59.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()